# Исследование надежности заемщиков

**Задача:** 
- исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

**Данные:**
- статистика о платёжеспособности клиентов банка

**Описание данных:**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## 1. Предобработка данных

### 1.1 Загрузка и изучение общей информации о данных 

In [1]:
# импорт необходимые библиотеки
import pandas as pd

In [2]:
# загрузка файла
data = pd.read_csv("D:\data_borrows.csv")

In [3]:
# изучение таблицы
data.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья


In [4]:
# изучение типов данных и пропущенных значений в столбцах
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# изучим пропущенные значения
data.isna().sum().sort_values(ascending=False)

days_employed       2174
total_income        2174
children               0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
purpose                0
dtype: int64

In [6]:
# определение количества дублирующих строк
data.duplicated().sum()

54

In [7]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод:**
- в таблице есть столбцы, в которых имеются пропущенные значения (days_employed и total_income)
- в таблице есть дублированные строки (54)
- в таблице есть столбцы, в которых некорректный тип данных (days_employed)
- в столбцах с days_employed и children имеются аномалии - отрицательные значения и большие значения

### 1.2 Работа с пропущенными и аномальными значениями

**Столбец total_income**

Столбец total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [8]:
median_df = data.groupby('income_type')['total_income'].median() 

for inc_type in data['income_type'].unique():
    data.loc[data['income_type'] == inc_type, 'total_income'] = data.loc[data['income_type'] == inc_type, 'total_income']\
    .fillna(median_df[inc_type])

**Столбец children**

In [9]:
# вывод переченя уникальных значений столбца children

sorted(data['children'].unique().tolist())

[-1, 0, 1, 2, 3, 4, 5, 20]

В столбце children есть два аномальных значения (-1 и 20). Удалим эти строки из таблицы, в которых встречаются такие аномальные значения.

In [10]:
# удаление строк с аномальными значениями в столбце children

data = data[(data['children'] >= 0) & (data['children'] <= 5)]

In [11]:
# проверка удаления

sorted(data['children'].unique().tolist())

[0, 1, 2, 3, 4, 5]

**Столбец days_employed**

Поскольку отрицательные значения в столбце days_employed - это может быть результат опечатки, то заменим все отрицательные значения положительными

In [12]:
# замена всех отрицательных значений положительными
data['days_employed'] = abs(data['days_employed'])

На сумму трудовой стаж сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [13]:
# заполнение пропусков в столбце days_employed медианными значениями по каждому типу занятости income_type

median_df = data.groupby('income_type')['days_employed'].median() 

for inc_type in data['income_type'].unique():
    data.loc[data['income_type'] == inc_type, 'days_employed'] = data.loc[data['income_type'] == inc_type, 'days_employed']\
    .fillna(median_df[inc_type])

In [14]:
# вывод медианного значения трудового стажа в днях для каждого типа занятости

data.groupby('income_type')['days_employed'].agg('median').sort_values(ascending=False)

income_type
безработный        366413.652744
пенсионер          365286.622650
в декрете            3296.759962
госслужащий          2689.137274
сотрудник            1572.328285
компаньон            1555.947387
студент               578.751554
предприниматель       520.848083
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) - аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

**Проверка заполнения пропущенных значений**

In [15]:
data.isna().sum().sort_values(ascending=False)

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 1.3 Работа с типами данных

In [16]:
# замена вещественного типа данных в столбце total_income на целочисленный

data['total_income'] = data['total_income'].astype('int')

In [17]:
# проверка исправления типа данных

data['total_income'].dtype

dtype('int32')

### 1.4 Работа с дубликатами

Ранее было выявлено, что в таблице имеются 54 строки-дубликаты, поэтому удалим их.

In [18]:
# удаление строк-дубликатов

data = data.drop_duplicates()

In [19]:
# проверка на удаление строк-дубликатов

data.duplicated().sum()

0

**Удалим возможное наличие разных регистров в столбцах со строковыми данными**

In [20]:
# создание функцию для приведения к нижнему регистру

def lowers(i):
    data[i] = data[i].str.lower()

In [21]:
# приведение к нижнему регистру столбцы со строковыми данными с помощью созданной функции

lowers('education')
lowers('family_status')
lowers('purpose')

### 1.5 Категоризация данных

**Выделение категорий людей в зависимости от уровня дохода**

Создадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

То есть присвоим каждому заемщику категорию в зависимости от его уровня дохода.

In [22]:
# создание функции для категоризации

def categorize_income(category):
    if 0 <= category <= 30000:
        return 'E'
    if 30001 <= category <= 50000:
        return 'D'
    if 50001 <= category <= 200000:
        return 'C'
    if 200001 <= category <= 1000000:
        return 'B'
    if 1000001 <= category >= 1000001:
        return 'A'

In [23]:
# проведение категоризации с помощью созданной функции

data['total_income_category'] = data['total_income'].apply(categorize_income)

In [24]:
# проверка, что категории созданы корректно

data[['total_income','total_income_category']].head(2)

,total_income,total_income_category
0,253875,B
1,112080,C


**Выделение категории людей в зависимости от цели кредита**

Определим, на какие цели люди берут кредит, чтобы можно было сделать категоризацию в зависимости от цели кредита

In [25]:
sorted(data['purpose'].unique().tolist())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [26]:
# создание функции для категоризации

def categorize_purpose(row):
    if 'автом' in row:
        return 'операции с автомобилем'
    if ('жил' in row) or ('недвиж' in row):
        return 'операции с недвижимостью'
    if 'свадьб'in row:
        return 'проведение свадьбы'
    if 'образов' in row:
        return 'получение образования'

In [27]:
# применение функции 

data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [28]:
# проверка, что категории созданы корректно

data.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем


In [29]:
# проверка, что категории созданы корректно

data[['purpose','purpose_category']].head(5)

,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,покупка жилья,операции с недвижимостью
3,дополнительное образование,получение образования
4,сыграть свадьбу,проведение свадьбы


**Выделение категории людей в зависимости от количества детей**

In [30]:
# создание функции для категоризации 

def categorize_children (child):
        if child == 0:
            return 'Бездетные'
        elif 1 <= child <= 2:
            return 'Имеют детей от 1 до 2'
        elif child >= 3:
            return 'Многодетные'

In [31]:
# применение функции

data['children_category'] = data['children'].apply(categorize_children)

In [32]:
# проверка, что категории созданы корректно

data[['children','children_category']].head(4)

,children,children_category
0,1,Имеют детей от 1 до 2
1,1,Имеют детей от 1 до 2
2,0,Бездетные
3,3,Многодетные


## 2. Исследовательский анализ данных 

### 2.1 Определение, есть зависимость между количеством детей и возвратом кредита в срок

In [33]:
# создание сводной таблицы, где будет отражена информация о проценте должников в зависимости от количества детей

children_and_credit_pivot = data.pivot_table(index = 'children_category', columns = 'debt', values = 'education_id',\
                                             aggfunc = 'count')
children_and_credit_pivot[0] = children_and_credit_pivot[0] + children_and_credit_pivot[1]
children_and_credit_pivot = children_and_credit_pivot.rename(columns = {1:'debt',0:'total_people'})
children_and_credit_pivot['ratio %'] = children_and_credit_pivot['debt']/children_and_credit_pivot['total_people'] * 100
children_and_credit_pivot['ratio %'] = round(children_and_credit_pivot['ratio %'],1)

children_and_credit_pivot.sort_values(by = 'ratio %',ascending = False)

debt,total_people,debt,ratio %
children_category,,,
Имеют детей от 1 до 2,6861,638,9.3
Многодетные,380,31,8.2
Бездетные,14107,1063,7.5


**Вывод**
* Больше всего должников в группе людей, у которых есть дети от 1 до 2. 
* Меньше всего должников в группе людей, у которых нет детей.
* Увеличение доли людей, которые не возвращают кредит в срок, не прямопорпорционально увеличению количества детей в семье.

### 2.2 Определение, есть ли зависимость между семейным положением и возвратом кредита в срок

In [34]:
family_and_credit_pivot = data.pivot_table(index = 'family_status', columns = 'debt', values = 'education_id', \
                                           aggfunc = 'count')
family_and_credit_pivot[0] = family_and_credit_pivot[0] + family_and_credit_pivot[1]
family_and_credit_pivot = family_and_credit_pivot.rename(columns = {1:'debt',0:'total_people'})
family_and_credit_pivot['ratio %'] = family_and_credit_pivot['debt']/family_and_credit_pivot['total_people'] * 100
family_and_credit_pivot['ratio %'] = round(family_and_credit_pivot['ratio %'],1)

family_and_credit_pivot.sort_values(by = 'ratio %',ascending = False)

debt,total_people,debt,ratio %
family_status,,,
не женат / не замужем,2796,273,9.8
гражданский брак,4146,385,9.3
женат / замужем,12266,927,7.6
в разводе,1189,84,7.1
вдовец / вдова,951,63,6.6


**Вывод:** 
* Больше всего должников в группе людей, которые не женаты.
* Меньше всего должников в группе людей, которые овдовели.

### 2.3 Определение, есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [35]:
income_and_credit_pivot = data.pivot_table(index = 'total_income_category', columns = 'debt', values = 'education_id', \
                                          aggfunc = 'count')
income_and_credit_pivot[0] = income_and_credit_pivot[0] + income_and_credit_pivot[1]
income_and_credit_pivot = income_and_credit_pivot.rename(columns = {1:'debt',0:'total_people'})
income_and_credit_pivot['ratio %'] = income_and_credit_pivot['debt']/income_and_credit_pivot['total_people'] * 100
income_and_credit_pivot['ratio %'] = round(income_and_credit_pivot['ratio %'],1)

income_and_credit_pivot.sort_values(by = 'ratio %',ascending = False)

debt,total_people,debt,ratio %
total_income_category,,,
E,22,2,9.1
C,15938,1353,8.5
A,25,2,8.0
B,5014,354,7.1
D,349,21,6.0


**Вывод:** 
* Больше всего должников в группе людей, у которых доход до 30 000 в мес.
* Меньше всего должников в группе людей, у которых доход от 50 001 до 200 000 в мес.
* Увеличение доли людей, которые не возвращают кредит в срок, не прямопропорционально уменьшению дохода у людей.

### 2.4 Определение, есть ли зависимость между целями кредита и возвратом кредита в срок

In [36]:
purpose_and_credit_pivot = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'education_id', \
                                            aggfunc = 'count')
purpose_and_credit_pivot[0] = purpose_and_credit_pivot[0] + purpose_and_credit_pivot[1]
purpose_and_credit_pivot = purpose_and_credit_pivot.rename(columns = {1:'debt',0:'total_people'})
purpose_and_credit_pivot['ratio_debt %'] = purpose_and_credit_pivot['debt']/purpose_and_credit_pivot['total_people'] * 100
purpose_and_credit_pivot['ratio_debt %'] = round(purpose_and_credit_pivot['ratio_debt %'],2)

purpose_and_credit_pivot.sort_values(by = 'ratio_debt %',ascending = False)

debt,total_people,debt,ratio_debt %
purpose_category,,,
операции с автомобилем,4281,400,9.34
получение образования,3989,369,9.25
проведение свадьбы,2324,183,7.87
операции с недвижимостью,10754,780,7.25


**Вывод:** 
* Больше всего должников среди тех, чья цель кредита - покупка автомобиля.
* Меньше всего должников среди тех, чья цель кредита - покупка недвижимости.

## 3. Общие выводы и рекомендации для выдачи кредита

### 3.1 Общие выводы

* Увеличение доли людей, которые не возвращают кредит в срок, не прямопропорционально уменьшению дохода у людей. Однако в срок отдают кредит больше те люди, которые не имеют детей.
* Увеличение доли людей, которые не возвращают кредит в срок, не прямопорпорционально увеличению количества детей в семье. Однако задерживаются с оплатой те люди, у которых доход не большой (меньше 30 тыс. в мес.).
* Рискованне всего давать кредит на покупку автомобиля, т.к. должников в этой категории больше.
* Рискованее всего давать кредит нежанатым людям, т.к. должников в этой категории больше.

### 3.2 Рекомендации для выдачи кредита

Первыми в очереди на получение кредита должны стать те люди, у которых:

- нет детей,
- которые овдовели,
- которые имеют доход от 50 тыс. до 200 тыс. в мес.,
- которые берут кредит с целью покупки недвижимости.